# Community Analyses through QIIME (Cohort 1)

11/09/2020<br>
Author: Robert Mills<br>
Environment: Qiime2-2019.7 & Qiime1 when specified<br> 
<br>
<i>This notebook contains code for reproducing several figures related to microbial community  relationships to IBD mediated through Qiime2 from the manuscript, <b> "Meta–omics Reveals Microbiome Driven Proteolysis as a Contributing Factor to Severity of Ulcerative Colitis Disease Activity" </b>by Mills et al. <br><br>
More specifically this notebook is a companion to a separate notebook for Cohort 2 QIIME analyses and has code for recreating analyses performed related to the UC cohort 1 as well as compiling beta-diversity associations to clinical metrics. <br><br>
Data sheets used here will be made available on Massive.ucsd.edu

In [2]:
#Import dependencies
%matplotlib inline

from os import mkdir
import os
import copy
from os.path import abspath, join as pjoin, exists
from shutil import copy2, move
from time import strftime, strptime
from numpy import nan, isnan, arange
from pandas import read_csv, Series, DataFrame
from IPython.display import Image
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

### Create Biom Files

In [ ]:
#Make subset of bacterial proteins only:
df = pd.read_csv("./2Search/CSVs/NormalizedCommonReps.txt", 
                 sep = '\t', index_col= "datarest$ProteinID")

#Remove any human derived proteins
df = df[df.index.str.contains('k99_') != False]
#df.to_csv('./2Search/CSVs/NormalizedCommonReps_nohuman.txt', sep = '\t')

In [7]:
#2 Search pDB Approach - no human
#Convert tab-separated file to biom file
!biom convert -i ./2Search/CSVs/NormalizedCommonReps_nohuman.txt \
-o ./2Search/NormalizedCommonReps2_nohuman.biom \
-m ../UC_MP_Emperor_Map_1.txt \
--table-type="OTU table" --to-hdf5

In [12]:
#2 Search pDB Approach
#Convert tab-separated file to biom file
!biom convert -i ./2Search/CSVs/NormalizedCommonReps.txt \
-o ./2Search/NormalizedCommonReps2.biom \
-m ../UC_MP_Emperor_Map_1.txt \
--table-type="OTU table" --to-hdf5

In [28]:
#Metabolomics
#Convert tab-separated file to biom file
!biom convert -i ../Metabolomics/Feature_table.txt \
-o ./Metabolomics.biom \
-m ../UC_MP_Emperor_Map_1.txt \
--table-type="OTU table" --to-hdf5

In [3]:
#16S
#Convert tab-separated file to biom file
!biom convert -i ../Genomics/16S/reference-hit_idswap2_blankremove.txt \
-o ./16S.biom \
-m ../UC_MP_Emperor_Map_1.txt \
--table-type="OTU table" --to-hdf5

In [1]:
#Serum
!biom convert -i ../Serum/CSVs/NormalizedCommonReps_ids.txt \
-o ./Serum_Common.biom \
-m ../UC_MP_Emperor_Map_1.txt \
--table-type="OTU table" --to-hdf5

In [2]:
#MG
!biom convert -i ../Genomics/Shotgun/Salmon_CPMs_0s.txt \
-o ./Salmon_CPMs.biom \
-m ../UC_MP_Emperor_Map_1.txt \
--table-type="OTU table" --to-hdf5

### Import all as Qiime2 artifacts

In [29]:
!qiime tools import \
  --input-path ./Metabolomics.biom \
  --type 'FeatureTable[Frequency]' \
  --output-path Metabolomics_biom.qza

In [14]:
!qiime tools import \
  --input-path ./2Search/NormalizedCommonReps2.biom\
  --type 'FeatureTable[Frequency]' \
  --output-path 2search_common_biom.qza

In [4]:
!qiime tools import \
  --input-path ./16S.biom\
  --type 'FeatureTable[Frequency]' \
  --output-path 16S_biom.qza

In [3]:
!qiime tools import \
  --input-path ./Salmon_CPMs.biom\
  --type 'FeatureTable[Frequency]' \
  --output-path ./Salmon_CPMs_biom.qza

In [4]:
!qiime tools import \
  --input-path ./Serum_Common.biom\
  --type 'FeatureTable[Frequency]' \
  --output-path ./Serum_Common_biom.qza

In [ ]:
!qiime tools import \
  --input-path ./2Search/NormalizedCommonReps2_nohuman.biom\
  --type 'FeatureTable[Frequency]' \
  --output-path ./pDB_Common_nohuman_biom.qza

### Feature table summarize

In [9]:
!qiime feature-table summarize \
  --i-table ./pDB_Common_nohuman_biom.qza \
  --o-visualization ./pDB_Common_nohuman_biom.qzv \
  --m-sample-metadata-file ../UC_MP_Emperor_Map.txt

Saved Visualization to: ./pDB_Common_nohuman_biom.qzv


In [30]:
!qiime feature-table summarize \
  --i-table Metabolomics_biom.qza \
  --o-visualization Metabolomics_biom.qzv \
  --m-sample-metadata-file ../UC_MP_Emperor_Map.txt

Saved Visualization to: Metabolomics_biom.qzv


In [17]:
!qiime feature-table summarize \
  --i-table 2search_common_biom.qza \
  --o-visualization 2search_biom.qzv \
  --m-sample-metadata-file ../UC_MP_Emperor_Map.txt

Saved Visualization to: 2search_biom.qzv


In [5]:
!qiime feature-table summarize \
  --i-table 16S_biom.qza \
  --o-visualization 16S_biom.qzv \
  --m-sample-metadata-file ../UC_MP_Emperor_Map.txt

Saved Visualization to: 16S_biom.qzv


In [5]:
!qiime feature-table summarize \
  --i-table ./Serum_Common_biom.qza \
  --o-visualization ./Serum_Common_biom.qzv \
  --m-sample-metadata-file ../UC_MP_Emperor_Map.txt

Saved Visualization to: ./Serum_Common_biom.qzv


In [6]:
!qiime feature-table summarize \
  --i-table ./Salmon_CPMs_biom.qza \
  --o-visualization Salmon_CPMs_biom.qzv \
  --m-sample-metadata-file ../UC_MP_Emperor_Map.txt

^C

Aborted!


In [ ]:
qiime feature-table summarize \
  --i-table ./Serum_biom.qza \
  --o-visualization Serum_biom.qzv \
  --m-sample-metadata-file ../../UC_MP_Emperor_Map.txt

In [45]:
!qiime feature-table summarize \
  --i-table ../Genomics/Shotgun/gOTU_table_UC40.qza \
  --o-visualization ../Genomics/gOTU_Table_UC40_Summary.qzv \
  --m-sample-metadata-file ../Genomics/16S/UC_Severity_1_MF_Idswap6.21.18.txt

Saved Visualization to: ../Genomics/gOTU_Table_UC40_Summary.qzv


Filter samples without histology data for quantitative assessment of the relationship between beta-diversity and histology

In [1]:
## remove samples without histology information 
!qiime feature-table filter-samples \
  --i-table ./gOTU_table_UC40_2.qza  \
  --m-metadata-file ../UC_Severity/UC_MP_Emperor_Map_v2_2020.txt \
  --p-where "Geboes_Grade_5_Numeric!='not applicable'" \
  --o-filtered-table ./gOTU_table_UC40_ForHist.qza

/bin/sh: qiime: command not found


##### Qiime 1 Beta-Diversity Statistics Adonis & PERMANOVA

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Amp.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_16S -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/16S/insertion_tree.relabelled.tre 

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_MG.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_MG -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/Shotgun/tree.nwk 

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Mb.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_Mb -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Mp.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_Mp -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Ser.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_Ser -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

For histology where some data was missing:

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Amp_ForHist/feature-table.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_16S_ForHist -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/16S/insertion_tree.relabelled.tre 

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_MG_ForHist/feature-table.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_MG_ForHist -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/Shotgun/tree.nwk

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Mp_ForHist/feature-table.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_Mp_ForHist -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Mb_ForHist/feature-table.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_Mb_ForHist -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../Cohort1_FinalTable_Ser_ForHist/feature-table.biom \
-m ../UC_MP_Emperor_Map_v2_2020_q2.txt \
-o ./Qiime/Cohort1_Ser_ForHist -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

For second cohort:

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/16S_CommonSamples_tubeid_UC/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_16S -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/16S/insertion_tree.relabelled.tre 

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/MB_CommonSamples_tubeid_UC/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_MB -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/MP_CommonSamples_tubeid_UC/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_MP -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/MG_CommonSamples_tubeid_UC/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_MG -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/Shotgun/tree.nwk 

Subset to only the UC samples with all quantitative variables filled out

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/16S_CommonSamples_tubeid_UC_quant/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_quant_16S -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/16S/insertion_tree.relabelled.tre 

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/MB_CommonSamples_tubeid_UC_quant/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_quant_MB -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/MP_CommonSamples_tubeid_UC_quant/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_quant_MP -p ../smallDB_correct/PCoA_Plots/paramaters2.txt

In [ ]:
!beta_diversity_through_plots.py -i ../../IBD200/MG_CommonSamples_tubeid_UC_quant/feature-table.biom \
-m ../../IBD200/Combined_metadata_inallomics_hist.txt \
-o ./Qiime/Cohort2_quant_MG -p ../smallDB_correct/PCoA_Plots/paramaters2_16S.txt -t ../Genomics/Shotgun/tree.nwk 

In [2]:
#Data types which require PERMANOVA or Adonis for quantitative data
Perma = ['sex','race','historic_extent','ASA_exposure', 'current_5ASA', 'steroid_exposure', 'current_steroids', 'IM_exposure', 'IM_type', 'biologic_exposure',
        'biologic_exposure_type','current_biologic','current_biologic_type','Experiment','TMT_Label']
Adonis = ['CRP','Calprotectin', 'partial_Mayo','age','age_diagnosis','disease_duration','height','stool_frequency','rectal_bleeding','PGA','mayo_endoscopic_score','UCEIS_endoscopic_score','COLLECTION_TIMESTAMP','Endoscopy_date']
Hist = ['Geboes_Grade_0_Numeric','Geboes_Grade_1_Numeric','Geboes_Grade_2A_Numeric','Geboes_Grade_2B_Numeric','Geboes_Grade_3_Numeric','Geboes_Grade_4_Numeric','Geboes_Grade_5_Numeric']

In [7]:
#For bray curtis metric:
dtypes=['Mp','Mb','16S','MG','Ser']
for i in dtypes:
    for j in Perma:
        !compare_categories.py --method permanova -i ./Qiime/Cohort1_$i/bray_curtis_dm.txt -m ../UC_MP_Emperor_Map_v2_2020_q2.txt -c $j -o ./Qiime/Permanova/Cohort1_$i/$j

In [8]:
#For unweighted unifrac metric:
dtypes=['16S','MG']
for i in dtypes:
    for j in Perma:
        !compare_categories.py --method permanova -i ./Qiime/Cohort1_$i/unweighted_unifrac_dm.txt -m ../UC_MP_Emperor_Map_v2_2020_q2.txt -c $j -o ./Qiime/Permanova/Cohort1_unweighted_unifrac_$i/$j

In [9]:
#Make subsets of the metadata to have only the columns of interest (Quantitative)
meta = pd.read_csv('../UC_MP_Emperor_Map_v2_2020_q2.txt',sep='\t',index_col='#SampleID')
meta[Adonis].to_csv('../UC_MP_Emperor_Map_v2_2020_q2_adonis.txt',sep='\t')
meta[Hist].to_csv('../UC_MP_Emperor_Map_v2_2020_q2_hist.txt',sep='\t')

In [ ]:
#For adonis using bray curtis metric:
dtypes=['Mp','Mb','16S','MG','Ser']
for i in dtypes:
    for j in Adonis:
        !compare_categories.py --method adonis -i ./Qiime/Cohort1_$i/bray_curtis_dm.txt -m ../UC_MP_Emperor_Map_v2_2020_q2_adonis.txt -c $j -o ./Qiime/Adonis/Cohort1_$i/$j

In [10]:
#For adonis using unweighted unifrac metric:
dtypes=['16S','MG']
for i in dtypes:
    for j in Adonis:
        !compare_categories.py --method adonis -i ./Qiime/Cohort1_$i/unweighted_unifrac_dm.txt -m ../UC_MP_Emperor_Map_v2_2020_q2_adonis.txt -c $j -o ./Qiime/Adonis/Cohort1_unweightedunifrac_$i/$j

In [12]:
#For bray curtis metric & histology:
dtypes=['Mp','Mb','16S','MG','Ser']
for i in dtypes:
    for j in Hist:
        !compare_categories.py --method adonis -i ./Qiime/Cohort1_ForHist_$i/bray_curtis_dm.txt -m ../UC_MP_Emperor_Map_v2_2020_q2_hist.txt -c $j -o ./Qiime/Adonis/Cohort1_$i/$j

In [10]:
#For unweighted unifrac metric & histology:
dtypes=['16S','MG']
for i in dtypes:
    for j in Hist:
        !compare_categories.py --method adonis -i ./Qiime/Cohort1_ForHist_$i/unweighted_unifrac_dm.txt -m ../UC_MP_Emperor_Map_v2_2020_q2_hist.txt -c $j -o ./Qiime/Adonis/Cohort1_unweightedunifrac_$i/$j

In [6]:
#Create a dictionary for all of the f and p-values
Dictionary={}
dtypes=['Mp','Mb','16S','MG','Ser']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    cats=[]
    for j in Perma:
        df=pd.read_csv('./Qiime/Permanova/Cohort1_%s/%s/permanova_results.txt' % (i,j),sep='\t',index_col='method name')
        pval=df.loc['p-value'][0][0:6]
        fval=df.loc['test statistic'][0][0:6]
        pvalues.append(pval)
        fvalues.append(fval)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_BrayCurtis' % i:pvalues,
                           'FStatistic_%s_BrayCurtis' % i :fvalues})
        Dictionary[i]=newdf

In [3]:
#Create a dictionary for all of the f and p-values (Unweighted UniFrac)
Dictionary2={}
dtypes=['16S','MG']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    cats=[]
    for j in Perma:
        df=pd.read_csv('./Qiime/Permanova/Cohort1_unweighted_unifrac_%s/%s/permanova_results.txt' % (i,j),sep='\t',index_col='method name')
        pval=df.loc['p-value'][0][0:6]
        fval=df.loc['test statistic'][0][0:6]
        pvalues.append(pval)
        fvalues.append(fval)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_UnweightedUniFrac' % i:pvalues,
                           'FStatistic_%s_UnweightedUniFrac' % i :fvalues})
        Dictionary2[i]=newdf

In [13]:
Categorical_df = Dictionary['Mp'].merge(Dictionary['16S'],left_on='Variable',right_on='Variable')
Categorical_df=Categorical_df.merge(Dictionary['Mb'],left_on='Variable',right_on='Variable')
Categorical_df=Categorical_df.merge(Dictionary['MG'],left_on='Variable',right_on='Variable')
Categorical_df=Categorical_df.merge(Dictionary['Ser'],left_on='Variable',right_on='Variable')
Categorical_df=Categorical_df.merge(Dictionary2['MG'],left_on='Variable',right_on='Variable')
Categorical_df=Categorical_df.merge(Dictionary2['16S'],left_on='Variable',right_on='Variable')
Categorical_df.index = Categorical_df['Variable']
Categorical_df.drop(columns='Variable',inplace=True)

In [14]:
#Create a dictionary for all of the f and p-values
Quantitative = Hist+Adonis
Dictionary={}
dtypes=['Mp','Mb','16S','MG','Ser']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    r2values=[]
    cats=[]
    for j in Quantitative:
        df=pd.read_csv('./Qiime/Adonis/Cohort1_%s/%s/adonis_results.txt' % (i,j),sep='\t')
        a=list(df.loc[5][0].split(' '))
        alist = ' '.join(a).split()
        pval=alist[6]
        fval=alist[4]
        r2=alist[5]
        pvalues.append(pval)
        fvalues.append(fval)
        r2values.append(r2)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_BrayCurtis' % i:pvalues,
                            'R2_%s_BrayCurtis' % i:r2values,
                           'FStatistic_%s_BrayCurtis' % i :fvalues})
        Dictionary[i]=newdf

In [15]:
#Create a dictionary for all of the f and p-values
Quantitative = Hist+Adonis
Dictionary2={}
dtypes=['16S','MG']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    r2values=[]
    cats=[]
    for j in Quantitative:
        df=pd.read_csv('./Qiime/Adonis/Cohort1_unweightedunifrac_%s/%s/adonis_results.txt' % (i,j),sep='\t')
        a=list(df.loc[5][0].split(' '))
        alist = ' '.join(a).split()
        pval=alist[6]
        fval=alist[4]
        r2=alist[5]
        pvalues.append(pval)
        fvalues.append(fval)
        r2values.append(r2)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_UnweightedUniFrac' % i:pvalues,
                            'R2_%s_UnweightedUniFrac' % i:r2values,
                           'FStatistic_%s_UnweightedUniFrac' % i :fvalues})
        Dictionary2[i]=newdf

In [16]:
Quant_df = Dictionary['Mp'].merge(Dictionary['16S'],left_on='Variable',right_on='Variable')
Quant_df=Quant_df.merge(Dictionary['Mb'],left_on='Variable',right_on='Variable')
Quant_df=Quant_df.merge(Dictionary['MG'],left_on='Variable',right_on='Variable')
Quant_df=Quant_df.merge(Dictionary['Ser'],left_on='Variable',right_on='Variable')
Quant_df=Quant_df.merge(Dictionary2['MG'],left_on='Variable',right_on='Variable')
Quant_df=Quant_df.merge(Dictionary2['16S'],left_on='Variable',right_on='Variable')
Quant_df.index = Quant_df['Variable']
Quant_df.drop(columns='Variable',inplace=True)

In [17]:
dtypes=['Mp','Mb','16S','MG','Ser']
for i in dtypes:
    Quant_df['%s'%i] = Quant_df['pval_%s_BrayCurtis'%i]+', '+Quant_df['FStatistic_%s_BrayCurtis'%i] +', '+Quant_df['R2_%s_BrayCurtis'%i]

In [19]:
dtypes=['16S','MG']
for i in dtypes:
    Quant_df['%s Unweighted UniFrac'%i] = Quant_df['pval_%s_UnweightedUniFrac'%i]+', '+Quant_df['FStatistic_%s_UnweightedUniFrac'%i] +', '+Quant_df['R2_%s_UnweightedUniFrac'%i]

In [36]:
dtypes=['Mp','Mb','16S','MG','Ser']
for i in dtypes:
    Categorical_df['%s'%i] = Categorical_df['pval_%s_BrayCurtis'%i]+', '+Categorical_df['FStatistic_%s_BrayCurtis'%i]

In [37]:
dtypes=['16S','MG']
for i in dtypes:
    Categorical_df['%s Unweighted UniFrac'%i] = Categorical_df['pval_%s_UnweightedUniFrac'%i]+', '+Categorical_df['FStatistic_%s_UnweightedUniFrac'%i]

In [20]:
totaldf=pd.concat([Quant_df,Categorical_df])

In [21]:
totaldf.to_csv('../UC_Cohort1_AdonisPermanova_Stats.csv')

Perform analyses for Second Cohort

In [ ]:
#Data types which require PERMANOVA categorical significance.
Categorical=['sex','Race','ASA_Exposure','Biologic_Exposure',
             'Biologic_Exposure_Type_IFX1_ADA2_GOL3_VDZ4_SIM5_UST6_TOF7',
             'Current_5ASA','Current_Antidepressents','Current_Biologic','Current_Biologic_Type_IFX1_ADA2_GOL3_VDZ4_SIM5_UST6_TOF7',
            'Current_IM','Current_IM_Type_1AZA_26MP_3MTX','Current_Steroids','Historic_Extent',
             'IM_Exposure','IM_Exposure_Type_1AZA_26MP_3MTX','Smoker','Steroid_Exposure']

Quantitative=['PGA','host_height','Rectal_Bleeding','Mayo_Endoscopic_Score','UCEIS','partial_Mayo','Age_at_Diagnosis','Disease_Duration',
             'Stool_Frequency','host_age','Geboes_Grade_0_Numeric','Geboes_Grade_1_Numeric','Geboes_Grade_2A_Numeric','Geboes_Grade_2B_Numeric','Geboes_Grade_3_Numeric','Geboes_Grade_4_Numeric','Geboes_Grade_5_Numeric']



In [ ]:
#For bray curtis metric:
dtypes=['MP','MB','16S','MG']
for i in dtypes:
    for j in Categorical:
        !compare_categories.py --method permanova -i ./Qiime/Cohort2_$i/bray_curtis_dm.txt -m ../../IBD200/Combined_metadata_inallomics_hist.txt -c $j -o ./Qiime/Permanova/Cohort2_$i/$j

In [ ]:
#For unifrac curtis metric:
dtypes=['16S','MG']
for i in dtypes:
    for j in Categorical:
        !compare_categories.py --method permanova -i ./Qiime/Cohort2_$i/unweighted_unifrac_dm.txt -m ../../IBD200/Combined_metadata_inallomics_hist.txt -c $j -o ./Qiime/Permanova/Cohort2_unweighted_unifrac_$i/$j

In [ ]:
#Create a dictionary for all of the f and p-values
Dictionary={}
dtypes=['MP','MB','16S','MG']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    cats=[]
    for j in Categorical:
        df=pd.read_csv('./Qiime/Permanova/Cohort2_%s/%s/permanova_results.txt' % (i,j),sep='\t',index_col='method name')
        pval=df.loc['p-value'][0][0:6]
        fval=df.loc['test statistic'][0][0:6]
        pvalues.append(pval)
        fvalues.append(fval)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_BrayCurtis' % i:pvalues,
                           'FStatistic_%s_BrayCurtis' % i :fvalues})
        Dictionary[i]=newdf

In [ ]:
Categorical_df = Dictionary['MP'].merge(Dictionary['16S'],left_on='Variable',right_on='Variable')
Categorical_df=Categorical_df.merge(Dictionary['MB'],left_on='Variable',right_on='Variable')
Categorical_df=Categorical_df.merge(Dictionary['MG'],left_on='Variable',right_on='Variable')
Categorical_df.index=Categorical_df['Variable']
Categorical_df=Categorical_df.drop(columns='Variable')

In [ ]:
#Create a dictionary for all of the f and p-values (Unweighted UniFrac)
Dictionary={}
dtypes=['16S','MG']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    cats=[]
    for j in Categorical:
        df=pd.read_csv('./Qiime/Permanova/Cohort2_unweighted_unifrac_%s/%s/permanova_results.txt' % (i,j),sep='\t',index_col='method name')
        pval=df.loc['p-value'][0][0:6]
        fval=df.loc['test statistic'][0][0:6]
        pvalues.append(pval)
        fvalues.append(fval)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_UnweightedUniFrac' % i:pvalues,
                           'FStatistic_%s_UnweightedUniFrac' % i :fvalues})
        Dictionary[i]=newdf

In [ ]:
Categorical_df2 = Dictionary['MG'].merge(Dictionary['16S'],left_on='Variable',right_on='Variable')
Categorical_df2.index=Categorical_df2['Variable']
Categorical_df2=Categorical_df2.drop(columns='Variable')



In [ ]:
#Save a metadata file that is all quantitative
df=pd.read_csv('../../IBD200/Combined_metadata_inallomics_hist.txt',sep='\t')
df.index=df['#SampleID']
df=df[df['Diagnosis']=='UC']
df=df[Quantitative]
for i in Quantitative:
    df=df[pd.to_numeric(df[i], errors='coerce').notnull()]

In [ ]:
#For bray curtis metric:
dtypes=['MP','MB','16S','MG']
for i in dtypes:
    for j in Quantitative:
        !compare_categories.py --method adonis -i ./Qiime/Cohort2_quant_$i/bray_curtis_dm.txt -m ../../IBD200/Combined_metadata_inallomics_hist_UC_quantitative.txt -c $j -o ./Qiime/Adonis/Cohort2_quant_$i/$j -n 999

In [ ]:
#For unweighted_unifrac metric:
dtypes=['16S','MG']
for i in dtypes:
    for j in Quantitative:
        !compare_categories.py --method adonis -i ./Qiime/Cohort2_quant_$i/unweighted_unifrac_dm.txt -m ../../IBD200/Combined_metadata_inallomics_hist_UC_quantitative.txt -c $j -o ./Qiime/Adonis/Cohort2_quant_unweightedunifrac_$i/$j -n 999

In [ ]:
#Create a dictionary for all of the f and p-values
Dictionary2={}
dtypes=['MP','MB','16S','MG']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    r2values=[]
    cats=[]
    for j in Quantitative:
        df=pd.read_csv('./Qiime/Adonis/Cohort2_quant_%s/%s/adonis_results.txt' % (i,j),sep='\t')
        a=list(df.loc[5][0].split(' '))
        alist = ' '.join(a).split()
        pval=alist[6]
        fval=alist[4]
        r2=alist[5]
        pvalues.append(pval)
        fvalues.append(fval)
        r2values.append(r2)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_BrayCurtis' % i:pvalues,
                            'R2_%s_BrayCurtis' % i:r2values,
                           'FStatistic_%s_BrayCurtis' % i :fvalues})
        Dictionary2[i]=newdf

In [ ]:
Quantitative_df = Dictionary2['MP'].merge(Dictionary2['16S'],left_on='Variable',right_on='Variable')
Quantitative_df=Quantitative_df.merge(Dictionary2['MB'],left_on='Variable',right_on='Variable')
Quantitative_df=Quantitative_df.merge(Dictionary2['MG'],left_on='Variable',right_on='Variable')
Quantitative_df.index = Quantitative_df['Variable']
Quantitative_df=Quantitative_df.drop(columns='Variable')

In [ ]:
#Create a dictionary for all of the f and p-values
Dictionary2={}
dtypes=['16S','MG']
for i in dtypes:
    pvalues=[]
    fvalues=[]
    r2values=[]
    cats=[]
    for j in Quantitative:
        df=pd.read_csv('./Qiime/Adonis/Cohort2_quant_unweightedunifrac_%s/%s/adonis_results.txt' % (i,j),sep='\t')
        a=list(df.loc[5][0].split(' '))
        alist = ' '.join(a).split()
        pval=alist[6]
        fval=alist[4]
        r2=alist[5]
        pvalues.append(pval)
        fvalues.append(fval)
        r2values.append(r2)
        cats.append(j)
        newdf=pd.DataFrame({'Variable':cats,
                            'pval_%s_UnweightedUniFrac' % i:pvalues,
                            'R2_%s_UnweightedUniFrac' % i:r2values,
                           'FStatistic_%s_UnweightedUniFrac' % i :fvalues})
        Dictionary2[i]=newdf

In [ ]:
#Combine the tables together
Quantitative_df2 = Dictionary2['MG'].merge(Dictionary2['16S'],left_on='Variable',right_on='Variable')
Quantitative_df2.index = Quantitative_df2['Variable']
Quantitative_df2=Quantitative_df2.drop(columns='Variable')
Quantitative_df3=Quantitative_df.merge(Quantitative_df2,left_index=True,right_index=True)
Categorical_df3=Categorical_df.merge(Categorical_df2,left_index=True,right_index=True)

In [ ]:
#Relabel them to be consistent between analyses
dtypes=['MP','MB','16S','MG']
for i in dtypes:
    Quantitative_df3['%s'%i] = Quantitative_df3['pval_%s_BrayCurtis'%i]+', '+Quantitative_df3['FStatistic_%s_BrayCurtis'%i] +', '+Quantitative_df3['R2_%s_BrayCurtis'%i]

dtypes=['16S','MG']
for i in dtypes:
    Quantitative_df3['%s Unweighted UniFrac'%i] = Quantitative_df3['pval_%s_UnweightedUniFrac'%i]+', '+Quantitative_df3['FStatistic_%s_UnweightedUniFrac'%i] +', '+Quantitative_df3['R2_%s_UnweightedUniFrac'%i]


dtypes=['MP','MB','16S','MG']
for i in dtypes:
    Categorical_df3['%s'%i] = Categorical_df3['pval_%s_BrayCurtis'%i]+', '+Categorical_df3['FStatistic_%s_BrayCurtis'%i]


dtypes=['16S','MG']
for i in dtypes:
    Categorical_df3['%s Unweighted UniFrac'%i] = Categorical_df3['pval_%s_UnweightedUniFrac'%i]+', '+Categorical_df3['FStatistic_%s_UnweightedUniFrac'%i]

In [ ]:
#Concatenate and save the statistics
totaldf=pd.concat([Quantitative_df3,Categorical_df3])
totaldf.to_csv('../UC_Cohort2_AdonisPermanova_Stats.csv')

##### Core metrics - Qiime2

In [48]:
#gOTUs
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ../../IBD200/Metagenomics/tree.qza \
  --i-table ../Genomics/Shotgun/gOTU_table_UC40.qza \
  --p-sampling-depth 16635 \
  --m-metadata-file ../Genomics/16S/UC_Severity_1_MF_Idswap6.21.18.txt \
  --output-dir core-metrics-results_MGgOTU

Saved FeatureTable[Frequency] to: core-metrics-results_MGgOTU/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: core-metrics-results_MGgOTU/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_MGgOTU/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_MGgOTU/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_MGgOTU/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results_MGgOTU/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results_MGgOTU/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results_MGgOTU/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results_MGgOTU/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results_MGgOTU/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results_MGgOTU/weig

In [32]:
!qiime feature-table summarize \
  --i-table ../Genomics/Shotgun/gOTU_table_UC40.qza \
  --o-visualization ./Metagenome_gOTU_UC.qzv \
  --m-sample-metadata-file ./UC_Severity_gOTU_metadata_2020.txt

Saved Visualization to: ./Metagenome_gOTU_UC.qzv


In [55]:
#MG
!biom convert -i ./gOTU_table_UC40_2.txt \
-o ./gOTU_table_UC40_2.biom \
-m ./UC_Severity_gOTUreg_metadata_2020_2.txt \
--table-type="OTU table" --to-hdf5

In [56]:
#Metagenomics using UniFrac. Based on centrifuge counts.
!qiime tools import \
  --input-path ./gOTU_table_UC40_2.biom \
  --type 'FeatureTable[Frequency]' \
  --output-path ./gOTU_table_UC40_2.qza

Imported ./gOTU_table_UC40_2.biom as BIOMV210DirFmt to ./gOTU_table_UC40_2.qza


In [10]:
#2Searh pDB using the no-human proteins
!qiime diversity core-metrics \
  --i-table pDB_Common_nohuman_biom.qza \
    --p-sampling-depth 539826 \
--m-metadata-file ../UC_MP_Emperor_Map.txt \
--output-dir core-metrics-results_pDB_noHuman

Saved FeatureTable[Frequency] to: core-metrics-results_pDB_noHuman/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_pDB_noHuman/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_pDB_noHuman/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_pDB_noHuman/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results_pDB_noHuman/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results_pDB_noHuman/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results_pDB_noHuman/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results_pDB_noHuman/bray_curtis_pcoa_results.qza
Saved Visualization to: core-metrics-results_pDB_noHuman/jaccard_emperor.qzv
Saved Visualization to: core-metrics-results_pDB_noHuman/bray_curtis_emperor.qzv


In [35]:
!qiime diversity core-metrics \
  --i-table 2search_common_biom.qza \
    --p-sampling-depth 694193 \
--m-metadata-file ../UC_MP_Emperor_Map.txt \
--output-dir core-metrics-results_2search

Saved FeatureTable[Frequency] to: core-metrics-results_2search/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_2search/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_2search/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results_2search/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results_2search/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results_2search/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results_2search/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results_2search/bray_curtis_pcoa_results.qza
Saved Visualization to: core-metrics-results_2search/jaccard_emperor.qzv
Saved Visualization to: core-metrics-results_2search/bray_curtis_emperor.qzv


For metabolomics data with 'phylogenetic' assessment of metabolites through Qemistree

In [30]:
!qiime tools import \
  --input-path ../Metabolomics/StandardizedMzMIne2020Rerun/FBMN-download_qza_table_data-mzminev20/clusterinfo_summary/library_identifications.tsv \
  --output-path ../Metabolomics/StandardizedMzMIne2020Rerun/FBMN-download_qza_table_data-mzminev20/library_identifications.qza \
  --type FeatureData[Molecules]

Imported ../Metabolomics/StandardizedMzMIne2020Rerun/FBMN-download_qza_table_data-mzminev20/clusterinfo_summary/library_identifications.tsv as TSVMoleculesFormat to ../Metabolomics/StandardizedMzMIne2020Rerun/FBMN-download_qza_table_data-mzminev20/library_identifications.qza


In [33]:
!qiime qemistree make-hierarchy \
  --i-csi-results ../Metabolomics/StandardizedMzMIne2020Rerun/GNPS_QEMISTREE/output_folder/fingerprints.qza \
  --i-feature-tables ../Metabolomics/StandardizedMzMIne2020Rerun/FBMN-download_qza_table_data-mzminev20/qiime2_output/qiime2_table.qza \
--i-library-matches ../Metabolomics/StandardizedMzMIne2020Rerun/FBMN-download_qza_table_data-mzminev20/library_identifications.qza \
--o-tree ../Metabolomics/StandardizedMzMIne2020Rerun/qemistree.qza \
  --o-feature-table ../Metabolomics/StandardizedMzMIne2020Rerun/feature-table-hashed.qza \
  --o-feature-data ../Metabolomics/StandardizedMzMIne2020Rerun/feature-data.qza

Saved Phylogeny[Rooted] to: ../Metabolomics/StandardizedMzMIne2020Rerun/qemistree.qza
Saved FeatureTable[Frequency] to: ../Metabolomics/StandardizedMzMIne2020Rerun/feature-table-hashed.qza
Saved FeatureData[Molecules] to: ../Metabolomics/StandardizedMzMIne2020Rerun/feature-data.qza


In [36]:
!qiime feature-table summarize \
  --i-table ../Metabolomics/StandardizedMzMIne2020Rerun/feature-table-hashed.qza \
  --o-visualization ../Metabolomics/StandardizedMzMIne2020Rerun/feature-table-hashed.qzv \
  --m-sample-metadata-file ../UC_MB_Emperor_Map.txt

Saved Visualization to: ../Metabolomics/StandardizedMzMIne2020Rerun/feature-table-hashed.qzv


In [37]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ../Metabolomics/StandardizedMzMIne2020Rerun/qemistree.qza \
  --i-table ../Metabolomics/StandardizedMzMIne2020Rerun/feature-table-hashed.qza \
  --p-sampling-depth 50000000 \
  --m-metadata-file ../UC_MB_Emperor_Map.txt \
  --output-dir core-metrics-results-MB-phylogenetic

Saved FeatureTable[Frequency] to: core-metrics-results-MB-phylogenetic/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: core-metrics-results-MB-phylogenetic/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results-MB-phylogenetic/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results-MB-phylogenetic/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results-MB-phylogenetic/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results-MB-phylogenetic/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results-MB-phylogenetic/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results-MB-phylogenetic/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results-MB-phylogenetic/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results-MB-phylogenetic/

In [34]:
!qiime qemistree get-classyfire-taxonomy \
  --i-feature-data ../Metabolomics/StandardizedMzMIne2020Rerun/feature-data.qza \
  --o-classified-feature-data ../Metabolomics/StandardizedMzMIne2020Rerun/classified-feature-data.qza

Saved FeatureData[Molecules] to: ../Metabolomics/StandardizedMzMIne2020Rerun/classified-feature-data.qza


##### Adonis statistics for main fig. Qiime2

In [ ]:
##Make a subset of the metadata to contain only numerics
df = pd.read_csv('../UC_MP_Emperor_Map.txt',sep='\t', index_col='id')
df['partial_Mayo'] = df['partial_Mayo'].astype('float')
numeric_cols = ['pielou_e','partial_Mayo']
df = df[numeric_cols]
df.to_csv('../UC_MP_Emperor_Map_numerics.txt',sep='\t')


In [13]:
!qiime diversity adonis \
  --i-distance-matrix ./core-metrics-results_2search/bray_curtis_distance_matrix.qza \
  --m-metadata-file ../UC_MP_Emperor_Map_numerics.txt \
  --o-visualization ./core-metrics-results_2search/bray_curtis_adonis_pmayo.qzv \
  --p-formula partial_Mayo

Saved Visualization to: ./core-metrics-results_2search/bray_curtis_adonis_pmayo.qzv


In [15]:
!qiime diversity adonis \
  --i-distance-matrix ../core-metrics-results_Serum/bray_curtis_distance_matrix.qza \
  --m-metadata-file ../UC_MP_Emperor_Map_numerics.txt \
  --o-visualization ../core-metrics-results_Serum/bray_curtis_adonis_pmayo.qzv \
  --p-formula partial_Mayo

Saved Visualization to: ../core-metrics-results_Serum/bray_curtis_adonis_pmayo.qzv


In [25]:
##Make a subset of the metadata to contain only numerics
df = pd.read_csv('../Genomics/16S/UC_Severity_1_MF_Idswap6.21.18.txt',sep='\t')
df.rename(columns={'#SampleID':'id'},inplace=True)
df.index = df['id']
df = df[df['partial_mayo']!='not applicable']

df['partial_mayo'] = df['partial_mayo'].astype('float')
numeric_cols = ['mayo_endoscopic_score','partial_mayo']
df = df[numeric_cols]
df.to_csv('../UC_MP_Emperor_Map_16Snumerics.txt',sep='\t')

In [26]:
!qiime diversity adonis \
  --i-distance-matrix ../Genomics/16S/core-metrics-results_idswap2_newmetadata_allsamples/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../UC_MP_Emperor_Map_16Snumerics.txt \
  --o-visualization ../Genomics/16S/core-metrics-results_idswap2_newmetadata_allsamples/unweighted_unifrac_adonis_pmayo.qzv \
  --p-formula partial_mayo

Saved Visualization to: ../Genomics/16S/core-metrics-results_idswap2_newmetadata_allsamples/unweighted_unifrac_adonis_pmayo.qzv


In [27]:
!qiime diversity adonis \
  --i-distance-matrix ../Genomics/16S/core-metrics-results_idswap2_newmetadata_allsamples/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../UC_MP_Emperor_Map_16Snumerics.txt \
  --o-visualization ../Genomics/16S/core-metrics-results_idswap2_newmetadata_allsamples/weighted_unifrac_adonis_pmayo.qzv \
  --p-formula partial_mayo

Saved Visualization to: ../Genomics/16S/core-metrics-results_idswap2_newmetadata_allsamples/weighted_unifrac_adonis_pmayo.qzv


In [38]:
##Make a subset of the metadata to contain only numerics
df = pd.read_csv('../UC_MB_Emperor_Map.txt',sep='\t', index_col='id')
df['partial_Mayo'] = df['partial_Mayo'].astype('float')
numeric_cols = ['pielou_e','partial_Mayo']
df = df[numeric_cols]
df.to_csv('../UC_MB_Emperor_Map_numerics.txt',sep='\t')

In [40]:
!qiime diversity adonis \
  --i-distance-matrix ./core-metrics-results-MB-phylogenetic/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../UC_MB_Emperor_Map_numerics.txt \
  --o-visualization ./core-metrics-results-MB-phylogenetic/weighted_unifrac_adonis_pmayo.qzv \
  --p-formula partial_Mayo

Saved Visualization to: ./core-metrics-results-MB-phylogenetic/weighted_unifrac_adonis_pmayo.qzv


In [41]:
!qiime diversity adonis \
  --i-distance-matrix ./core-metrics-results-MB-phylogenetic/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../UC_MB_Emperor_Map_numerics.txt \
  --o-visualization ./core-metrics-results-MB-phylogenetic/unweighted_unifrac_adonis_pmayo.qzv \
  --p-formula partial_Mayo

Saved Visualization to: ./core-metrics-results-MB-phylogenetic/unweighted_unifrac_adonis_pmayo.qzv


In [42]:
!qiime diversity adonis \
  --i-distance-matrix ./core-metrics-results-MB-phylogenetic/bray_curtis_distance_matrix.qza \
  --m-metadata-file ../UC_MB_Emperor_Map_numerics.txt \
  --o-visualization ./core-metrics-results-MB-phylogenetic/bray_curtis_adonis_pmayo.qzv \
  --p-formula partial_Mayo

Saved Visualization to: ./core-metrics-results-MB-phylogenetic/bray_curtis_adonis_pmayo.qzv


In [49]:
!qiime diversity adonis \
  --i-distance-matrix ./core-metrics-results_MGgOTU/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../UC_MP_Emperor_Map_16Snumerics.txt \
  --o-visualization ./core-metrics-results_MGgOTU/weighted_unifrac_adonis_pmayo.qzv \
  --p-formula partial_mayo

Saved Visualization to: ./core-metrics-results_MGgOTU/weighted_unifrac_adonis_pmayo.qzv


In [50]:
!qiime diversity adonis \
  --i-distance-matrix ./core-metrics-results_MGgOTU/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../UC_MP_Emperor_Map_16Snumerics.txt \
  --o-visualization ./core-metrics-results_MGgOTU/unweighted_unifrac_adonis_pmayo.qzv \
  --p-formula partial_mayo

Saved Visualization to: ./core-metrics-results_MGgOTU/unweighted_unifrac_adonis_pmayo.qzv


In [51]:
!qiime diversity adonis \
  --i-distance-matrix ./core-metrics-results_MGgOTU/bray_curtis_distance_matrix.qza \
  --m-metadata-file ../UC_MP_Emperor_Map_16Snumerics.txt \
  --o-visualization ./core-metrics-results_MGgOTU/bray_curtis_adonis_pmayo.qzv \
  --p-formula partial_mayo

Saved Visualization to: ./core-metrics-results_MGgOTU/bray_curtis_adonis_pmayo.qzv


##### Create a combined table for random forest analyses

In [20]:
Mb = pd.read_csv('../Metabolomics/StandardizedMzMIne2020Rerun/UC40_MB_Table_normalized.csv')
Mb

,Unnamed: 0,H8,H11,H12,L7,H14,H16,L22,H7,L9,...,H5,H1,L12,H18,L20,L14,L11,H17,H3,H19
0,1,46909.498401,0.000000,0.000000,179.819380,0.000000,0.000000,0.000000,531.088310,1271.684763,...,0.000000,599.695089,0.000000,0.000000,6128.969926,6013.327664,320.286032,0.000000,0.000000,0.000000
1,2,21456.423961,0.000000,3209.658048,0.000000,1443.415289,4484.421327,0.000000,0.000000,0.000000,...,0.000000,707.640558,26628.566164,614.571046,0.000000,0.000000,0.000000,10999.273694,628.479395,3588.391143
2,4,17150.358235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,508.676267,552.002760,...,0.000000,3055.094548,0.000000,0.000000,360.907039,0.000000,0.000000,0.000000,0.000000,0.000000
3,5,18506.034913,12964.227060,50320.039626,24622.798409,8791.566438,48346.760347,10000.295012,46644.925805,25199.698602,...,14888.980413,13836.875906,8250.907422,18713.108934,26692.233282,33902.363004,0.000000,3862.980372,0.000000,29333.622313
4,6,14764.662583,0.000000,0.000000,45.464297,0.000000,0.000000,0.000000,760.312761,400.923157,...,0.000000,250.920502,0.000000,0.000000,0.000000,936.729145,0.000000,0.000000,0.000000,0.000000
5,7,11933.452966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,934.435196,308.408944,...,0.000000,224.047768,0.000000,0.000000,10731.423898,4938.744773,0.000000,0.000000,0.000000,0.000000
6,8,23509.327949,238.960875,2872.267196,21287.137541,751.349323,1674.558384,2593.909401,23874.233192,1004.766215,...,1005.563649,6042.383248,12235.184253,14661.388796,10894.567164,15239.497263,13510.629037,114.861375,15047.233221,1153.853265
7,9,14812.638646,0.000000,0.000000,5070.186710,10615.555566,245.209067,13709.412682,0.000000,429.369600,...,6358.672761,36255.702427,0.000000,9214.322778,96.097618,0.000000,222.506894,0.000000,39531.430739,5617.084413
8,10,14547.208020,0.000000,0.000000,4735.698207,1727.315911,0.000000,6699.919421,0.000000,536.245370,...,670.011213,19070.538821,0.000000,8106.818511,36.107493,0.000000,43.697930,0.000000,16210.632852,892.785097
9,11,11838.586875,28659.377981,0.000000,0.000000,241.303695,781.786822,2326.596671,0.000000,1510.519681,...,0.000000,664.197731,0.000000,1689.391530,15051.337939,1171.765579,409.760146,0.000000,1447.331765,0.000000


In [21]:
#Create combined table with all data
Mb = pd.read_csv('../Metabolomics/StandardizedMzMIne2020Rerun/UC40_MB_Table_normalized.csv', index_col = '#OTU ID')
Mp = pd.read_csv('../pDB_Proteomics/2Search/CSVs/NormalizedCommonReps.txt', sep = '\t', index_col = 'datarest$ProteinID')
Ser = pd.read_csv('../Serum/CSVs/NormalizedCommonReps_ids.txt', sep = '\t', index_col = 'datarest$ProteinID')
MG = pd.read_csv('./gOTU_table_UC40_2.txt', sep = '\t', index_col = '#OTU ID')
Amp = pd.read_csv('../Genomics/16S/reference-hit_idswap2_noblanks.txt', sep = '\t', index_col = '#OTU ID')


#Overlapping IDS are a problem, so append the metaproteome ids with _MP to signify metaproteome.
Mp.index = Mp.index + '_MP'

#Concatenate data
Objs = [Mb, Mp, Ser, MG, Amp]
alldf = pd.concat(Objs)

alldf.index.rename('Features', inplace = True)

#Save the new feature table
alldf.to_csv('./allfeaturesconcat3.txt', sep = '\t')

/Users/rhmills/miniconda3/envs/qiime2-2019.1/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [22]:
#Create biom file
!biom convert -i ./allfeaturesconcat3.txt \
-o ../Allfeaturesconcat3.biom \
-m ../UC_MP_Emperor_Map_1.txt \
--table-type="OTU table" --to-hdf5

In [23]:
!qiime tools import \
  --input-path ../Allfeaturesconcat3.biom \
  --type 'FeatureTable[Frequency]' \
  --output-path ../Allfeaturesconcat3.qza

Imported ../Allfeaturesconcat3.biom as BIOMV210DirFmt to ../Allfeaturesconcat3.qza


<i> Performed random forest analyses on supercomputer using the Allfeaturesconcat.biom file </i>

##### 16S analysis - Qiime2

In [2]:
# Initializes the notebook with inline display
%matplotlib inline

from os import mkdir
import os
import copy
from os.path import abspath, join as pjoin, exists
from shutil import copy2, move
from time import strftime, strptime
from numpy import nan, isnan, arange
from pandas import read_csv, Series, DataFrame
from IPython.display import Image
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import FileLinks, FileLink

#### Import all files as Qiime2 artifacts

In [6]:
!qiime tools import \
  --input-path ../Genomics/16S/reference-hit_idswap2.biom \
  --type 'FeatureTable[Frequency]' \
  --output-path biom_id2.qza

Imported ../Genomics/16S/reference-hit_idswap2.biom as BIOMV210DirFmt to biom_id2.qza


In [ ]:
!qiime feature-table summarize \
  --i-table biom_id2.qza \
  --o-visualization biom_id2.qzv \
  --m-sample-metadata-file ../Genomics/16S/UC_Severity_1_MF_Idswap6.21.18.txt

In [ ]:
## rep-seqs 
!qiime tools import \
  --input-path reference-hit.seqs.fa \
  --output-path sequences.qza \
  --type 'FeatureData[Sequence]'

In [ ]:
!qiime alignment mafft \
  --i-sequences sequences.qza \
  --o-alignment aligned-rep-seqs.qza

In [ ]:
!qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza

In [ ]:
!qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza

In [ ]:
!qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

#### Core Diversity Analysis

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table biom_id2.qza \
  --p-sampling-depth 4166 \
  --m-metadata-file UC_Severity_1_MF_Idswap6.21.18.txt \
  --output-dir core-metrics-results_idswap2_newmetadata_allsamples

#### Alpha diversity analyses

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results_idswap2/faith_pd_vector.qza \
  --m-metadata-file UC_Severity_1_MF_Idswap6.21.18.txt \
  --o-visualization core-metrics-results_idswap2_newmetadata/faith-pd-group-significance.qzv

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results_idswap2/evenness_vector.qza \
  --m-metadata-file UC_Severity_1_MF_Idswap6.21.18.txt \
  --o-visualization core-metrics-results_idswap2_newmetadata/evenness-group-significance.qzv

#### Taxanomic analysis

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads sequences.qza \
  --o-classification taxonomy.qza

In [ ]:
!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

In [ ]:
!qiime taxa barplot \
  --i-table biom_id2.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file UC_Severity_1_MF_Idswap6.21.18.txt \
  --o-visualization taxa-bar-plots_idswap2.qzv